In [ ]:
import sacct
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
import dask.dataframe as dd
from random import randint
import numpy as np
from dask.array import from_array as fa

sourceFile = 'df_sacct.parquet'  # Source of sacct data if using offline mode.

In [ ]:
df = dd.read_csv(sourceFile)
# df = dd.read_parquet(sourceFile)  # For if you're smart and use parquet files instead of csvs

In [ ]:
pd.set_option('display.max.columns', None)
pd.set_option('display.width', 150)
print(df.shape,'\n\n')
print(df.head(),'\n\n')
df.describe()

In [ ]:
# Example of populating empty column with new data

runTimes = df['CPUTimeRaw'].compute().tolist()

limitExamples = []
for time in runTimes:
    newTime = round((time+randint(0,100)/10)*(1+randint(0,100)/100))
    limitExamples.append(newTime)

print(runTimes[:10])
print(limitExamples[:10])
df['Timelimit'] = fa(limitExamples)
print(df['Timelimit'])
dd.to_csv(df,'df_sacctFix.csv')

In [ ]:
# Time wasted query
timeWaste = []
timeEff = []
for ind, value in enumerate(df['CPUTimeRaw'].compute().tolist()):
    timeWaste.append(df['Timelimit'].compute().tolist()[ind] - value)
    #timeEff.append(round(100*(value/df['Timelimit'].compute().tolist()[ind])))


In [ ]:
print(df['CPUTimeRaw'].compute().tolist()[:10])
print(df['Timelimit'].compute().tolist()[:10])
print((timeWaste)[:10])

In [ ]:
# Load in an old query rather than build a new one
queryDF = pd.read_parquet('sacctQuery.parquet')

In [ ]:
# Construct dictionary and pandas df with old and new data to display query
#newCatDir = {'CPUTimeRaw':df['CPUTimeRaw'].compute().tolist(), 'TimeLimit':df['Timelimit'].compute().tolist(), 'TimeWaste': timeWaste}
newCatDir = {'JobIDRaw': df['JobIDRaw'],'CPUTimeRaw': df['CPUTimeRaw'], 'TimeLimit': df['Timelimit'], 'TimeWaste': timeWaste}#, 'TimeEff%': timeEff}

queryDF = pd.DataFrame(newCatDir)

In [ ]:
# Display query dataframe preview
queryDF.head(50)

In [ ]:
queryDF.describe()

In [ ]:
# Sorting query df
queryDF.sort_values('TimeWaste', ascending=False)

In [ ]:
# Query save system: If desired, users can save their current query as a parquet file for use later

saveName = 'sacctQuery'  # Doesn't need to include the .parqet or .csv extension
queryDF.to_parquet(saveName+'.parquet')
